In [2]:
pip install opencv-python opencv-contrib-python


  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/76/9e/4d47a69d3338a00c51535028d25a7d1a17c4762e4a46ae6a577f495de252/opencv_python-4.10.0.82-cp37-abi3-win_amd64.whl.metadata
  Obtaining dependency information for opencv-contrib-python from https://files.pythonhosted.org/packages/3a/98/fd10252d16ac2202c52e646b051e89ad4c9f884df47508bdcf312ae1929a/opencv_contrib_python-4.10.0.82-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB 1.3 MB/s eta 0:00:31
   ---------------------------------------- 0.1/38.8 MB 1.2 MB/s eta 0:00:33
   ---------------------------------------- 0.2/38.8 MB 1.9 MB/s eta 0:00:21
   ---------------------------------------- 0.3/38.8 MB 1.8 MB/s eta 0:00:21
   ---------------------------------------- 0.4/38.8 MB 2.0 MB/s eta 0:00:20
    --------------------------------------- 0.7/38.8 MB 2.7 MB/s eta 0:00:15
   - ---

In [21]:
import cv2
import numpy as np
import csv


def rotationMatrixToEulerAngles(R):
    sy = np.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])
    singular = sy < 1e-6

    if not singular:
        x = np.arctan2(R[2, 1], R[2, 2])
        y = np.arctan2(-R[2, 0], sy)
        z = np.arctan2(R[1, 0], R[0, 0])
    else:
        x = np.arctan2(-R[1, 2], R[1, 1])
        y = np.arctan2(-R[2, 0], sy)
        z = 0

    return np.degrees(x), np.degrees(y), np.degrees(z)

# Load the ArUco dictionary
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_1000)
parameters = cv2.aruco.DetectorParameters()

# Define the camera matrix and distortion coefficients
# Replace these placeholders with actual calibration values
camera_matrix = np.array([[1000, 0, 640], [0, 1000, 360], [0, 0, 1]], dtype=np.float32)  # Example values
dist_coeffs = np.array([0.1, -0.25, 0.001, 0.001, 0.1], dtype=np.float32)  # Example values

# ArUco marker size in meters
marker_length = 0.05  # Example: 5 cm

# Initialize video capture
cap = cv2.VideoCapture('challenge1.mp4')

# Prepare to write results to a CSV file
csv_file = open('output.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Frame ID', 'QR id', 'QR 2D', 'QR 3D'])

frame_id = 0

# Video writer for annotated output
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Detect ArUco markers
    corners, ids, rejected = cv2.aruco.detectMarkers(frame, aruco_dict, parameters=parameters)
    
    if ids is not None:
        # Estimate pose of each marker
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, marker_length, camera_matrix, dist_coeffs)
        
        for i in range(len(ids)):
            rvec, tvec = rvecs[i][0], tvecs[i][0]
            id = ids[i][0]
            corner = corners[i].reshape(4, 2).tolist()
            
            # Draw the marker and its axis
            cv2.aruco.drawDetectedMarkers(frame, corners)
            cv2.drawFrameAxes(frame, camera_matrix, dist_coeffs, rvec, tvec, marker_length * 0.5)
            
            # Calculate yaw, pitch, and roll from rvec
            R, _ = cv2.Rodrigues(rvec)
            yaw, pitch, roll = rotationMatrixToEulerAngles(R)
            
            # Write to CSV
            csv_writer.writerow([frame_id, id, corner, [np.linalg.norm(tvec), yaw, pitch, roll]])
    
    # Write the annotated frame to the output video
    out.write(frame)
    
    frame_id += 1
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything
cap.release()
out.release()
cv2.destroyAllWindows()
csv_file.close()


In [ ]:
#Same code with documentation

In [24]:
import cv2
import numpy as np
import csv

# Function to convert a rotation matrix to Euler angles
def rotationMatrixToEulerAngles(R):
    # Calculate the sy value
    sy = np.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])
    # Check for singularity
    singular = sy < 1e-6

    if not singular:
        # Calculate the angles
        x = np.arctan2(R[2, 1], R[2, 2])
        y = np.arctan2(-R[2, 0], sy)
        z = np.arctan2(R[1, 0], R[0, 0])
    else:
        # Handle the singularity case
        x = np.arctan2(-R[1, 2], R[1, 1])
        y = np.arctan2(-R[2, 0], sy)
        z = 0

    # Convert the angles from radians to degrees
    return np.degrees(x), np.degrees(y), np.degrees(z)

# Load the ArUco dictionary that contains the markers we will detect
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_1000)
# Create detector parameters
parameters = cv2.aruco.DetectorParameters()

# Define the camera matrix and distortion coefficients
# These should be replaced with actual calibration values
camera_matrix = np.array([[1000, 0, 640], [0, 1000, 360], [0, 0, 1]], dtype=np.float32)  # values for drawing x,y,z on the aruco in the video
dist_coeffs = np.array([0.1, -0.25, 0.001, 0.001, 0.1], dtype=np.float32)

# Define the size of the ArUco marker in meters
marker_length = 0.05  # aruco marker size

# Initialize video capture from the provided video file
cap = cv2.VideoCapture('challenge2.mp4')

# Open a CSV file to write the results
csv_file = open('output3.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)
# Write the header row
csv_writer.writerow(['Frame ID', 'QR id', 'QR 2D', 'QR 3D'])

frame_id = 0

# Initialize the video writer for the annotated output video
fourcc = cv2.VideoWriter_fourcc(*'XVID')

'''
Params of VideoWriter:
video output name , codec , frame rate - FPS , width of the frames , height of the frames
'''
out = cv2.VideoWriter('output5.avi', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

# Process the video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Detect ArUco markers in the frame
    corners, ids, rejected = cv2.aruco.detectMarkers(frame, aruco_dict, parameters=parameters)
    
    if ids is not None:
        # Estimate the pose of each detected marker
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, marker_length, camera_matrix, dist_coeffs)
        
        for i in range(len(ids)):
            # Get the rotation and translation vectors for each marker
            rvec, tvec = rvecs[i][0], tvecs[i][0]
            id = ids[i][0]
            # Get the 2D corner points of the marker
            corner = corners[i].reshape(4, 2).tolist()
            
            # Draw the detected markers on the frame
            cv2.aruco.drawDetectedMarkers(frame, corners)
            # Draw the axes of the detected marker
            cv2.drawFrameAxes(frame, camera_matrix, dist_coeffs, rvec, tvec, marker_length * 0.5)
            
            # Convert the rotation vector to a rotation matrix
            R, _ = cv2.Rodrigues(rvec)
            # Calculate the yaw, pitch, and roll angles
            yaw, pitch, roll = rotationMatrixToEulerAngles(R)
            
            # Write the data to the CSV file
            csv_writer.writerow([frame_id, id, corner, [np.linalg.norm(tvec), yaw, pitch, roll]])
            
            # Print the ID of the marker on the frame in the corner
            # Choose one corner of the marker to place text
            corner_to_print = corner[0]  # Choose one of the corners (e.g., top-left corner)
            text_offset_x = 10  # Offset from the corner in X direction
            text_offset_y = 10  # Offset from the corner in Y direction
            text_pos = (int(corner_to_print[0] + text_offset_x), int(corner_to_print[1] + text_offset_y))
            cv2.putText(frame, str(id), text_pos, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Write the annotated frame to the output video
    out.write(frame)
    
    frame_id += 1
    
    # Check for user input to quit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and writer objects
cap.release()
out.release()
cv2.destroyAllWindows()
# Close the CSV file
csv_file.close()


In [ ]:
# for camera_matrix dist_coeffs Calibration - if needed

In [ ]:
import cv2
import numpy as np

# Define the chessboard size
chessboard_size = (7, 6)

# Arrays to store object points and image points
obj_points = []
img_points = []

# Prepare object points (0,0,0), (1,0,0), (2,0,0), ..., (6,5,0)
objp = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)

# List of images for calibration
images = ['calib1.jpg', 'calib2.jpg', 'calib3.jpg', ...]

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

    if ret:
        img_points.append(corners)
        obj_points.append(objp)

# Calibrate the camera
ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, gray.shape[::-1], None, None)

# Print the obtained camera matrix and distortion coefficients
print("Camera Matrix:\n", camera_matrix)
print("Distortion Coefficients:\n", dist_coeffs)
